# Run time

## Setup

In [15]:
import os
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf

## Daten von Excel

in diesem Fall laden wir die Daten direkt aus der Excel Datei, aber das wir von euch ersetzt.

In [474]:
xlsx_path = '/home/pmf/MachineLearning/projects/PM10_MW1_Graz_2015-2019.xlsx'
df = pd.read_excel(xlsx_path)[170:210]

In [476]:
df.isna().sum()

Station:     0
Graz-DB      0
Graz-M G     0
Graz-S       0
Graz-O P     0
Graz-N       2
Graz-W       0
Lustb.      40
dtype: int64

# Vorhersage - ab hier zu verwenden

**df** ist das von euch zu übergebende DataFrame 

In [477]:
date_time = pd.to_datetime(df.pop('Station:'), format='%d.%m.%Y %H:%M:%S')
df = df.astype('float')

In [478]:
df = df[['Graz-N']]

In [479]:
timestamp_s = date_time.map(datetime.datetime.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

Shape muss gleich sein!

In [480]:
df = df[-48:]

# Normieren

In [481]:
df_norm = df

In [482]:
file1 = open("EA2_values.txt","r") 
mean_and_sdt = file1.read().split('\n')
len(mean_and_sdt)//2
train_mean = [float(x.split(' ')[-1]) for x in  mean_and_sdt[:len(mean_and_sdt)//2]]
train_std = [float(x.split(' ')[-1]) for x in  mean_and_sdt[len(mean_and_sdt)//2:]]
file1.close()

In [483]:
df_norm = (df_norm - train_mean) / train_std

In [484]:
df_norm = df_norm.interpolate(method='linear')

# Vorhersage

In [487]:
model = tf.keras.models.load_model('EA2-Modell/')

In [488]:
vorhersage = pd.DataFrame(model.predict(np.array([tf.constant(df_norm)]))[0])

In [489]:
vorhersage = vorhersage * train_std[0] + train_mean[0]

In [490]:
vorhersage

,0
0,17.550524
